# 1 - Load and Split Data

In [2]:
# import 
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.linear_model import  LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import  SVR
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb                                             
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import SCORERS, mean_absolute_percentage_error, mean_squared_error


In [3]:
# Load Data
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/lavibula/ML20222.PredictionBitcoin/main/data/data.csv")
# data = pd.read_csv(r"C:\Users\Administrator\OneDrive - Hanoi University of Science and Technology\ITE10 - Data Science and AI - HUST\20222\ML\Source_Codes\ML20222.PredictionBitcoin\data\data.csv")
# print(data.info())
data = data.sort_values('Date', ascending=True).reset_index(drop=True) # drop=True bỏ 'index' col cũ. 

# Use BTC_close_tomorrow as y (Target col) of X_today, instead of BTC_close_today
data["BTC_tomorrow"] = data["BTC_close"].shift(-1)
data = data.drop('BTC_close', axis=1) # drop BTC_close_today
data = data.iloc[:-1] # data = data.drop(data.index[-1])

print(data.info())
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2660 entries, 0 to 2659
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    2660 non-null   object 
 1   BTC_open                2660 non-null   float64
 2   BTC_high                2660 non-null   float64
 3   BTC_low                 2660 non-null   float64
 4   difficulty              2660 non-null   int64  
 5   addresses_active_count  2660 non-null   int64  
 6   sum_lock_weight         2660 non-null   int64  
 7   mean_lock_size_ytes     2660 non-null   float64
 8   total_fees_usd          2660 non-null   float64
 9   mean_hash_rate          2660 non-null   float64
 10  xfer_cnt                2660 non-null   int64  
 11  mean_tx_size_usd        2660 non-null   float64
 12  ETH                     2660 non-null   float64
 13  LTC                     2660 non-null   float64
 14  XRP                     2660 non-null   

,Date,BTC_open,BTC_high,BTC_low,difficulty,addresses_active_count,sum_lock_weight,mean_lock_size_ytes,total_fees_usd,mean_hash_rate,...,LTC,XRP,DOGE,COPPER,GOLD,SILVER,SPX,JP225,DJI,BTC_tomorrow
0,2016-03-10,412.8,417.5,410.3,158427203767,445273,426654988,6.881532e+05,1.731272e+04,1.220696e+06,...,0.00,0.00000,0.000000,0.0000,0.00,0.000,0.00,0.0,0.0,419.1
1,2016-03-11,415.8,422.4,415.1,158427203767,434658,398582424,6.227850e+05,1.710193e+04,1.260073e+06,...,0.00,0.00000,0.000000,0.0000,0.00,0.000,0.00,0.0,0.0,410.4
2,2016-03-12,419.1,420.7,407.0,158427203767,374730,331208848,5.750154e+05,1.398444e+04,1.134066e+06,...,0.00,0.00000,0.000000,0.0000,0.00,0.000,0.00,0.0,0.0,412.4
3,2016-03-13,410.4,415.9,409.6,158427203767,421585,334817852,6.293569e+05,1.460678e+04,1.047436e+06,...,0.00,0.00000,0.000000,0.0000,0.00,0.000,0.00,0.0,0.0,414.3
4,2016-03-14,412.4,416.1,411.2,158427203767,451902,437739524,7.199663e+05,1.689298e+04,1.197070e+06,...,0.00,0.00000,0.000000,0.0000,0.00,0.000,0.00,0.0,0.0,415.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,2023-06-17,26341.3,26767.3,26183.5,52350439455487,863600,559141195,1.724510e+06,1.041701e+06,3.643296e+08,...,76.87,0.47940,0.062193,0.0000,0.00,0.000,0.00,0.0,0.0,26339.7
2656,2023-06-18,26515.0,26679.3,26290.6,52350439455487,883864,603064705,1.985675e+06,7.946708e+05,3.929555e+08,...,77.20,0.48699,0.062107,3.8738,1969.45,24.198,0.00,0.0,0.0,26845.9
2657,2023-06-19,26339.7,27029.7,26295.1,52350439455487,920552,567091224,1.758290e+06,8.514319e+05,3.695343e+08,...,77.51,0.49341,0.062429,3.8643,1964.05,24.062,0.00,0.0,0.0,28307.7
2658,2023-06-20,26845.9,28393.0,26665.5,52350439455487,951926,543133484,1.677592e+06,1.052364e+06,3.539202e+08,...,80.31,0.49270,0.063108,3.8835,1947.70,23.234,4388.71,33155.0,34356.0,29996.9


In [4]:
# Split data for pre-train
from sklearn.model_selection import train_test_split

X = data.drop(['BTC_tomorrow', 'Date'], axis=1)
y = data['BTC_tomorrow']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.15, random_state=42, shuffle=False)

# 2 - Use best parameters for model.

In [ ]:
rf_params = {'bootstrap': True,
          'max_depth': None,
          'max_features': None,
          'min_samples_leaf': 1,
          'min_samples_split': 2,
          'n_estimators': 300}

lgb_params = {'subsample': 1.0,
 'random_state': 42,
 'objective': 'regression',
 'num_leaves': 35,
 'min_child_samples': 5,
 'metric': 'rmse',
 'max_depth': 8,
 'max_bin': 500,
 'learning_rate': 0.48,
 'lambda_l2': 0.2,
 'lambda_l1': 0.1,
 'boosting_type': 'dart'}


In [6]:
from sklearn.metrics import r2_score

models= [
LinearRegression(),
RandomForestRegressor(**rf_params)

]

entries = []                                           
for model in models:

    rmse_l = []
    mape_l = []
    r2_score_l = []
    
    model_name = model.__class__.__name__
    model.fit(X_train, y_train)
    y_pred_test = model.predict(X_test)

    r2_score_value = round(r2_score(y_test, y_pred_test), 4)
    rmse = round(mean_squared_error(y_test, y_pred_test, squared=False), 4)
    mape = mean_absolute_percentage_error(y_test, y_pred_test)
    mape = round(mape*100, 4)

    r2_score_l.append(r2_score_value)
    rmse_l.append(rmse)
    mape_l.append(mape)

    entries.append([model_name, np.mean(rmse_l), np.mean(mape_l), np.mean(r2_score_l)])

model_df = pd.DataFrame(entries, columns=['model_name', 'RMSE', 'MAPE%', 'r2_score'])
model_df.sort_values(by=['r2_score'], ascending=False)


,model_name,RMSE,MAPE%,r2_score
0,LinearRegression,1022.9105,3.3885,0.9412
1,RandomForestRegressor,1488.5628,5.5398,0.8755


In [7]:
total= pd.concat([model_df, df2],ignore_index=True)
total.sort_values(by=['score'], ascending=False)

NameError: name 'df2' is not defined